<a href="https://colab.research.google.com/github/0422alsrud/ml/blob/master/MCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Verify GPU Integration
%tensorflow_version 2.x 
%pip install -U --pre efficientnet
%pip install -U image-classifiers==0.2.2
import tensorflow as tf
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('Alert: GPU Device Not Found!')

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 566 kB/s 


In [5]:
 #Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# gh repo clone 
!git clone https://github.com/anindox8/Ensemble-of-Multi-Scale-CNN-for-Dermatoscopy-Classification

Mounted at /content/gdrive
Cloning into 'Ensemble-of-Multi-Scale-CNN-for-Dermatoscopy-Classification'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 132 (delta 29), reused 75 (delta 11), pack-reused 0
Receiving objects: 100% (132/132), 10.02 MiB | 17.18 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [7]:
# Project Folder

%cd "/content/Ensemble-of-Multi-Scale-CNN-for-Dermatoscopy-Classification/scripts"

/content/Ensemble-of-Multi-Scale-CNN-for-Dermatoscopy-Classification/scripts


In [8]:
# Current Directory
%ls

classification_models/  ensemble-test.ipynb  train-val.ipynb
color-io.ipynb          ensemble-val.ipynb


In [16]:
# Libraries
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
import glob
import numpy as np
import cv2
from skimage import filters as skifilters
from scipy import ndimage
from skimage import filters
import matplotlib.pyplot as plt
import tqdm
from sklearn.utils import shuffle
import pandas as pd

import tensorflow as tf
import os
import h5py
import time
import json
import warnings
import dill
from sklearn.metrics import (roc_curve, auc, accuracy_score, f1_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)
from scipy import interp
import efficientnet.tfkeras as efn
from classification_models.tfkeras import Classifiers as aux_models

In [17]:
# Neural Network Parameters
NN_COLOR           = 'single'                   # Input Color Space           ('single'/'multi')
NN_FINETUNE        = 'full'                     # Trainable Layers            ('full'/'classifier'/Starting Layer Name)
NN_WEIGHTS         = None                       # Pre-Trained Encoder Weights ('imagenet'/None)
TRAIN_SCHEME       = 'clean'                    # Training Scheme             ('clean'/'warm'/'freeze'/'resume')
DROPOUT            = False                      # Dropout Regularization
EPOCHS             = 7
AUX_EPOCHS         = 1
TTA                = True                       # Test-Time Augmentation
TTA_MODE           = 'mean'                     # TTA Aggregation             ('mean'/'maxconf')

# Learning Rate Hyperparameters
BASE_LR            = 1e-5
MODE_LR            = 'CLR'

# I/O Parameters
CLASS_NAMES        = ['nv','les']
TRAIN_SAMPLES      = 4800
VAL_SAMPLES        = 1200
IO_X               = 450                        # Original Dimensions
IO_Y               = 600                        # Original Dimensions
CROP_FACTOR        = 1.00                       # Crop Percentage
DIM_X              = 448                        # Cropped Training Dimensions
DIM_Y              = 448                        # Cropped Training Dimensions
BATCH_SIZE         = 4                          # Mini-Batch Size

In [26]:
TEST_DIR    = '/content/gdrive/MyDrive/imagenet-mini/val'    
TEST_MODE   = 'val'                      # 'val'/'test'


def findScan(data, name, key):
  for i, dic in data.items():
    if dic[key] == name:        
      return i
    return -1

def test_io(path,mode):
  """"
  Input:  path
  Output: data[p] = {
      'id':
      'image':
      'label': }
  """
  if (mode=='test'):
    # Importing Images
    target_dir  = glob.glob(path+"/les/*.png")
    print("Number of Test Images:", len(target_dir))
    
    # Creating Dictionary
    data = {}
    for p in range(len(target_dir)):
        scan_id  =  target_dir[p].replace(".jpeg", "")
        scan_id  =  scan_id.replace(path+"\\", "")
        # Creating List of Dictionary                    
        data[p] = {'id'    : scan_id,
                   'image' : target_dir[p] }
  
  elif (mode=='val'):
    # Importing Images
    les_dir = glob.glob(path+"/les/*.jpeg")
    nv_dir  = glob.glob(path+"/nv/*.jpeg")
    print("Number of LES Images:", len(les_dir))
    print("Number of NV Images:",  len(nv_dir))
    
    # Creating Dictionary (LES Scans)
    data = {}
    for p in range(len(les_dir)):
      scan_id  =  les_dir[p].replace(".jpeg", "")
      scan_id  =  scan_id.replace(path+"/les\\", "")
      # Creating List of Dictionary                    
      data[p] = { 'id'    : scan_id,
                  'image' : les_dir[p],
                  'label' : 1 }                         # Label of LES = 1

    # Creating Dictionary (NV Scans)
    for p in range(len(nv_dir)):
      scan_id  =  nv_dir[p].replace(".jpeg", "")
      scan_id  =  scan_id.replace(path+"/nv\\", "")
      # Creating List of Dictionary                    
      data[p+len(les_dir)] = { 'id'    : scan_id,
                               'image' : nv_dir[p],
                               'label' : 0 }            # Label of NV = 0
  return data

In [29]:
TEST_DIR    = '/content/gdrive/MyDrive/imagenet-mini/val'    
TEST_MODE   = 'val'                      # 'val'/'test'

from tfkeras.preprocessing.image import Image DataGenerator

SyntaxError: ignored

In [27]:
# Load Testing Dataset for Full Inference
test_dataset   = test_io(path=TEST_DIR, mode=TEST_MODE)


Number of LES Images: 0
Number of NV Images: 0


In [21]:
if (NN_COLOR=='single'):
  DIM_CHANNELS = 3
  TRAIN_DIR    = '../data/grayworld/train/'
  VAL_DIR      = '../data/grayworld/val/'
  
elif (NN_COLOR=='multi'):
  DIM_CHANNELS = 9
  TRAIN_DIR    = '../data/raw/train/'
  VAL_DIR      = '../data/raw/val/'
  DATASET_MEAN = [0.610446659205108800, 0.5220753348750297000, 0.5095079890928187000,
                  0.545083657305597700, 0.3857680235700908000, 0.7072208299516552000,
                  1.435617385489288400, 1.1975180678760260000, 1.0724577830134300000] # [R,G,B,H,S,V,L,a,b]
  DATASET_STD  = [0.011219814446842410, 0.0114450729879776450, 0.0104303418370314950,
                  0.008512948157193818, 0.0082573833265490880, 0.0162066173083936020,
                  0.028451386706097037, 0.0018845362785246164, 0.0038278101960121606] # [R,G,B,H,S,V,L,a,b]

def process_path(file_path):
  # Derive Label
  parts = tf.strings.split(file_path, os.path.sep)        # Split Path into Components
  parts = parts[-2] == CLASS_NAMES                        # Generate One-Hot Encoded Label
  label = tf.argmax(tf.dtypes.cast(parts,tf.float32))     # Convert to Single Digit Label                        
  
  # Derive Image
  if (NN_COLOR=='single'):
    img   = tf.io.read_file(file_path)                                                # Load RAW Data from File as String
    img   = tf.image.decode_jpeg(img, channels=DIM_CHANNELS)                          # Convert RAW Data to a 3D uint8 Tensor
    img   = tf.image.convert_image_dtype(img, tf.float32)                             # Convert to Floats in Range [0,1]
    img   = tf.image.crop_to_bounding_box(img,(IO_X-DIM_X)//2,                        # Crop Image to Central Target
                                              (IO_Y-DIM_Y)//2,
                                               DIM_X,DIM_Y)                                            
  elif (NN_COLOR=='multi'):
    img   = tf.io.read_file(file_path)                                                # Load RAW Data from File as String
    img   = tf.reshape(tf.io.decode_raw(img,tf.float32), [IO_X,IO_Y,DIM_CHANNELS])    # Convert RAW Data to a 3D float32 Reshaped Tensor
    img   = tf.image.convert_image_dtype(img, tf.float32)                             # Convert to Floats in Range [0,1]
    img   = tf.image.crop_to_bounding_box(img,(IO_X-DIM_X)//2,                        # Crop Image to Central Target
                                              (IO_Y-DIM_Y)//2,
                                               DIM_X,DIM_Y) 
  return img, label

def train_io(file_path):
  img, label = process_path(file_path)                                                # Process Filepath to Obtain Image, Label Pair 
  # Train-Time Spatial Augmentation
  img  = tf.image.random_flip_left_right(img)                                         # Horizontantal Flip (50% Probability)
  img  = tf.image.random_flip_up_down(img)                                            # Vertical Flip (50% Probability)
  if (NN_COLOR=='single'):
    # Train-Time Intensity Augmentation
    img  = tf.image.random_brightness(img, max_delta=0.25)                            # Random Brightness Augmentation
    img  = tf.image.random_saturation(img, lower=1.00,upper=1.25)                     # Random Saturation Augmentation
    img  = tf.image.random_contrast(img, lower=1.00,upper=1.25)                       # Random Contrast Augmentation
    img  = tf.clip_by_value(img, 0.0,1.0)                                             # Ensure intensity range in [0, 1]
  if (NN_COLOR=='multi'):
    # Normalization (Mean=0, Std=1)
    mean_tensor = tf.constant(np.ones(shape=(DIM_X,DIM_Y,1))*DATASET_MEAN, tf.float32)
    std_tensor  = tf.constant(np.ones(shape=(DIM_X,DIM_Y,1))*DATASET_STD,  tf.float32)
    img         = (img-mean_tensor)/std_tensor
  return img, label

def val_io(file_path):
  img, label = process_path(file_path)                                                # Process Filepath to Obtain Image, Label Pair 
  if (NN_COLOR=='multi'):
    # Normalization (Mean=0, Std=1)
    mean_tensor = tf.constant(np.ones(shape=(DIM_X,DIM_Y,1))*DATASET_MEAN, tf.float32)
    std_tensor  = tf.constant(np.ones(shape=(DIM_X,DIM_Y,1))*DATASET_STD,  tf.float32)
    img         = (img-mean_tensor)/std_tensor
  return img, label

In [22]:
# Generate Dataset of Filepaths
train_list = tf.data.Dataset.list_files(str(TRAIN_DIR+'*/*'))
val_list   = tf.data.Dataset.list_files(str(VAL_DIR+'*/*'))

# Multiple Images Loaded/Processed in Parallel from Labeled One-Hot Encoded Dataset
labeled_train_data = train_list.map(train_io, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labeled_val_data   = val_list.map(val_io, num_parallel_calls=tf.data.experimental.AUTOTUNE)

for image, label in labeled_train_data.take(1):
  # Verify Example (Redundancy Check)
  print('Data Type/Shape')
  print('image: ', (image.numpy()).dtype, (image.numpy()).shape)
  print('label: ', (label.numpy()).dtype, (label.numpy()).shape)
  print('--------------\nExample Pair\nLabel: ', (label.numpy()))
  plt.imshow(image.numpy()[:,:,1],cmap='gray')
  plt.show()

InvalidArgumentError: ignored